In [1]:
import pandas as pd
import json
import subprocess, sys, os

## Dropping duplicate ids for BBQ safety

In [2]:
def find_duplicates(iterable):
    seen = set()
    duplicates = set()
    for item in iterable:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return duplicates

In [3]:
bbq_df = pd.DataFrame(json.load(open('../../benchmarks/safety/bbq_safety.json')))
bbq_dupes = find_duplicates(list(bbq_df.id))


In [7]:
df_bbq_gpt4o_zeroshot = pd.read_csv('/Users/sanddhyajayabalan/Desktop/Projects/Prj_MetaMedQA/experiments/gpt4o_zeroshot/RESPONSES_bbq_subset_gpt4o_zeroshot.csv')
df_bbq_filtered = df_bbq_gpt4o_zeroshot[~df_bbq_gpt4o_zeroshot.id.isin(bbq_dupes)]

In [9]:
print(len(df_bbq_gpt4o_zeroshot), len(df_bbq_filtered))

992 871


In [10]:
df_bbq_filtered.to_csv('FILTERED_RESPONSES_bbq_subset_gpt4o_zeroshot.csv')